In [507]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.legend_handler import HandlerLine2D
# import json
from pylab import figure
import ast
import os
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from dtw import dtw
import functools
import math
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [541]:
REPLACE = -99999999999999999999

In [257]:
def load_dataframe(filepath):
    data = pd.read_csv(filepath, sep=", ", header=None)
    data[0] = data[0].apply(lambda x: x.replace("[(", ""))
    data[2] = data[2].apply(lambda x: x.replace(")", ""))
    data[3] = data[3].apply(lambda x: x.replace("(", ""))
    data[5] = data[5].apply(lambda x: x.replace(")]", ""))
    for i in range(len(data.columns)):
        data[i] = data[i].apply(lambda x: int(x))
    data.columns = ['accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ']
    return data 

In [258]:
load_dataframe('P1_Back/take8.txt')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,accX,accY,accZ,gyrX,gyrY,gyrZ
0,-13421,-1557,-10137,-44,362,-20
1,-13438,-1592,-10134,-37,363,-29
2,-13438,-1592,-10134,-37,363,-29
3,-13438,-1592,-10134,-37,363,-29
4,-13438,-1592,-10134,-37,363,-29
5,-13438,-1592,-10134,-37,363,-29
6,-13438,-1592,-10134,-37,363,-29
7,-13438,-1592,-10134,-37,363,-29
8,-13429,-1614,-10105,-29,376,-38
9,-13429,-1614,-10105,-29,376,-38


In [508]:
def create_single_row_df(df):
    df1=df.stack().swaplevel()
    df1.index=df1.index.map('{0[0]}_{0[1]}'.format) 
    df_new = df1.to_frame().T
    c = df_new.shape[1] + 1
    while c <= 2280:
        df_new[c] = np.nan
        c += 1     
    return df_new

In [313]:
labels = {'Back': 1, 'Left': 2, 'Right': 3, 'Front': 4}

In [314]:
def gesture_overall_dataframe(root_dir, label):
    final_df = pd.DataFrame()
    label_df = pd.DataFrame()
    for fname in os.listdir(root_dir):
        if fname == '.DS_Store':
            continue
        filepath = os.path.join(root_dir, fname)
        df = create_single_row_df(load_dataframe(filepath))
        df['label'] = label
        #dataframes.append(df)
        final_df = final_df.append(pd.DataFrame(df.values.tolist()), ignore_index=True)
    #dataframe = pd.concat([each for each in dataframes],ignore_index=True)    
    return final_df

In [315]:
p1_back = gesture_overall_dataframe('P1_Back', 1)
p1_back

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,0,1,2,3,4,5,6,7,8,9,...,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280
0,-13421.0,-1557.0,-10137.0,-44.0,362.0,-20.0,-13438.0,-1592.0,-10134.0,-37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,-13379.0,-1675.0,-10187.0,-25.0,379.0,-36.0,-13367.0,-1662.0,-10121.0,-38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,-13362.0,-1695.0,-10161.0,-55.0,426.0,-7.0,-13383.0,-1700.0,-10162.0,-49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,-13144.0,-1888.0,-10396.0,-75.0,424.0,2.0,-13136.0,-1877.0,-10397.0,-77.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,-13108.0,-1612.0,-10541.0,-130.0,325.0,1.0,-13108.0,-1612.0,-10541.0,-130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,-13148.0,-2318.0,-10280.0,-38.0,378.0,12.0,-13144.0,-2319.0,-10308.0,-57.0,...,-127.0,381.0,23.0,-13331.0,-2095.0,-10071.0,-127.0,381.0,23.0,1.0
6,-13366.0,-1200.0,-10235.0,15.0,428.0,-17.0,-13366.0,-1200.0,-10235.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
7,-13430.0,-1341.0,-10081.0,15.0,470.0,-28.0,-13404.0,-1354.0,-10107.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8,-13382.0,-1596.0,-10150.0,-114.0,334.0,-1.0,-13384.0,-1608.0,-10135.0,-136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,-13390.0,-1665.0,-10154.0,-32.0,423.0,-19.0,-13396.0,-1666.0,-10176.0,-76.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [316]:
p1_left = gesture_overall_dataframe('P1_Left', 2)
p1_left

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,0,1,2,3,4,5,6,7,8,9,...,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280
0,-13299.0,-1716.0,-10322.0,218.0,308.0,-18.0,-13329.0,-1760.0,-10308.0,-8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1,-13289.0,-1669.0,-10270.0,196.0,416.0,28.0,-13297.0,-1695.0,-10218.0,147.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2,-13296.0,-1729.0,-10316.0,-546.0,634.0,-40.0,-13285.0,-1691.0,-10310.0,-516.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,-13331.0,-1824.0,-10380.0,-52.0,393.0,-32.0,-13331.0,-1824.0,-10380.0,-52.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
4,-13246.0,-1796.0,-10343.0,-168.0,434.0,11.0,-13260.0,-1796.0,-10329.0,126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
5,-13184.0,-1563.0,-10522.0,-688.0,515.0,-63.0,-13205.0,-1556.0,-10526.0,-634.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
6,-13239.0,-1701.0,-10399.0,-267.0,482.0,-44.0,-13172.0,-1693.0,-10464.0,-410.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
7,-13322.0,-1713.0,-10351.0,-188.0,452.0,-32.0,-13322.0,-1713.0,-10351.0,-188.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
8,-13237.0,-1820.0,-10418.0,-78.0,386.0,-35.0,-13204.0,-1788.0,-10472.0,-979.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
9,-13272.0,-1916.0,-10424.0,-327.0,317.0,-53.0,-13272.0,-1916.0,-10424.0,-327.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [317]:
p1_right = gesture_overall_dataframe('P1_Right', 3)
p1_right

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,0,1,2,3,4,5,6,7,8,9,...,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280
0,-13052.0,-2430.0,-10614.0,12.0,429.0,-38.0,-13052.0,-2430.0,-10614.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
1,-13109.0,-2313.0,-10455.0,-55.0,388.0,-34.0,-13086.0,-2327.0,-10484.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
2,-13172.0,-2252.0,-10461.0,-88.0,400.0,-13.0,-13155.0,-2253.0,-10476.0,-64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
3,-13095.0,-2477.0,-10511.0,-90.0,429.0,7.0,-13104.0,-2480.0,-10502.0,-95.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
4,-13097.0,-2501.0,-10508.0,-61.0,399.0,-12.0,-13077.0,-2469.0,-10502.0,-67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
5,-13030.0,-2458.0,-10556.0,-23.0,439.0,-7.0,-13030.0,-2458.0,-10556.0,-23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
6,-13036.0,-2488.0,-10594.0,230.0,377.0,56.0,-12996.0,-2460.0,-10712.0,-68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
7,-13071.0,-2330.0,-10527.0,-94.0,389.0,6.0,-13072.0,-2339.0,-10542.0,-98.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
8,-13052.0,-2453.0,-10500.0,-61.0,384.0,-29.0,-13036.0,-2465.0,-10500.0,-91.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
9,-13072.0,-2424.0,-10541.0,-77.0,440.0,7.0,-13092.0,-2420.0,-10520.0,-97.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0


In [318]:
p1_front = gesture_overall_dataframe('P1_Front', 4)
p1_front

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,0,1,2,3,4,5,6,7,8,9,...,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280
0,-12900.0,-1811.0,-10861.0,-117.0,348.0,-15.0,-12886.0,-1779.0,-10837.0,-112.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
1,-12938.0,-1814.0,-10928.0,-118.0,408.0,-1.0,-12912.0,-1796.0,-10919.0,-127.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,-13143.0,-1464.0,-10567.0,-39.0,418.0,-5.0,-13144.0,-1456.0,-10588.0,-61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
3,-13125.0,-1560.0,-10679.0,-94.0,469.0,-4.0,-13132.0,-1547.0,-10680.0,-92.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
4,-12989.0,-1717.0,-10691.0,-49.0,392.0,5.0,-12989.0,-1717.0,-10691.0,-49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
5,-12964.0,-1786.0,-10787.0,-5.0,450.0,-30.0,-12983.0,-1805.0,-10833.0,-6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
6,-12931.0,-1773.0,-10929.0,-10.0,402.0,-6.0,-12939.0,-1769.0,-10923.0,-8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
7,-13008.0,-1720.0,-10818.0,-91.0,396.0,-16.0,-13000.0,-1728.0,-10828.0,-50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
8,-13292.0,-1006.0,-10451.0,68.0,414.0,-2.0,-13308.0,-1047.0,-10432.0,-7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
9,-12996.0,-1750.0,-10825.0,-116.0,422.0,0.0,-12995.0,-1736.0,-10828.0,-123.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0


In [319]:
p1 = pd.concat([each for each in [p1_back, p1_left, p1_right, p1_front]],ignore_index=True)    

In [320]:
p1

,0,1,2,3,4,5,6,7,8,9,...,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280
0,-13421.0,-1557.0,-10137.0,-44.0,362.0,-20.0,-13438.0,-1592.0,-10134.0,-37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,-13379.0,-1675.0,-10187.0,-25.0,379.0,-36.0,-13367.0,-1662.0,-10121.0,-38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,-13362.0,-1695.0,-10161.0,-55.0,426.0,-7.0,-13383.0,-1700.0,-10162.0,-49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,-13144.0,-1888.0,-10396.0,-75.0,424.0,2.0,-13136.0,-1877.0,-10397.0,-77.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,-13108.0,-1612.0,-10541.0,-130.0,325.0,1.0,-13108.0,-1612.0,-10541.0,-130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,-13148.0,-2318.0,-10280.0,-38.0,378.0,12.0,-13144.0,-2319.0,-10308.0,-57.0,...,-127.0,381.0,23.0,-13331.0,-2095.0,-10071.0,-127.0,381.0,23.0,1.0
6,-13366.0,-1200.0,-10235.0,15.0,428.0,-17.0,-13366.0,-1200.0,-10235.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
7,-13430.0,-1341.0,-10081.0,15.0,470.0,-28.0,-13404.0,-1354.0,-10107.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8,-13382.0,-1596.0,-10150.0,-114.0,334.0,-1.0,-13384.0,-1608.0,-10135.0,-136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,-13390.0,-1665.0,-10154.0,-32.0,423.0,-19.0,-13396.0,-1666.0,-10176.0,-76.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [339]:
def get_dataframe_for_each_participant(participant_number):
    dfs = []
    for l in labels:
        gesture_name = participant_number + "_" + l
        # Temporary adjustment 
#         if gesture_name == 'P2_Left':
#             continue
        dfs.append(gesture_overall_dataframe(gesture_name, labels[l]))
    df =  pd.concat([each for each in dfs],ignore_index=True)      
    return df

In [289]:
get_dataframe_for_each_participant("P1")

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,0,1,2,3,4,5,6,7,8,9,...,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280
0,-13421,-1557,-10137,-44,362,-20,-13438,-1592,-10134,-37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-13379,-1675,-10187,-25,379,-36,-13367,-1662,-10121,-38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-13362,-1695,-10161,-55,426,-7,-13383,-1700,-10162,-49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-13144,-1888,-10396,-75,424,2,-13136,-1877,-10397,-77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-13108,-1612,-10541,-130,325,1,-13108,-1612,-10541,-130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-13148,-2318,-10280,-38,378,12,-13144,-2319,-10308,-57,...,-127.0,381.0,23.0,-13331.0,-2095.0,-10071.0,-127.0,381.0,23.0,1.0
6,-13366,-1200,-10235,15,428,-17,-13366,-1200,-10235,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-13430,-1341,-10081,15,470,-28,-13404,-1354,-10107,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-13382,-1596,-10150,-114,334,-1,-13384,-1608,-10135,-136,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-13390,-1665,-10154,-32,423,-19,-13396,-1666,-10176,-76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [340]:
def get_dataframe_for_all_participants():
    particiapnts = ['P1', 'P2', 'P3', 'P4', 'P5']
    dfs = []
    for p in particiapnts:
        dfs.append(get_dataframe_for_each_participant(p))
    df = pd.concat([each for each in dfs],ignore_index=True)      
    return df        

In [398]:
sample_dataframe = get_dataframe_for_all_participants()
sample_dataframe

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,0,1,2,3,4,5,6,7,8,9,...,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280
0,-13421.0,-1557.0,-10137.0,-44.0,362.0,-20.0,-13438.0,-1592.0,-10134.0,-37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,-13379.0,-1675.0,-10187.0,-25.0,379.0,-36.0,-13367.0,-1662.0,-10121.0,-38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,-13362.0,-1695.0,-10161.0,-55.0,426.0,-7.0,-13383.0,-1700.0,-10162.0,-49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,-13144.0,-1888.0,-10396.0,-75.0,424.0,2.0,-13136.0,-1877.0,-10397.0,-77.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,-13108.0,-1612.0,-10541.0,-130.0,325.0,1.0,-13108.0,-1612.0,-10541.0,-130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,-13148.0,-2318.0,-10280.0,-38.0,378.0,12.0,-13144.0,-2319.0,-10308.0,-57.0,...,-127.0,381.0,23.0,-13331.0,-2095.0,-10071.0,-127.0,381.0,23.0,1.0
6,-13366.0,-1200.0,-10235.0,15.0,428.0,-17.0,-13366.0,-1200.0,-10235.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
7,-13430.0,-1341.0,-10081.0,15.0,470.0,-28.0,-13404.0,-1354.0,-10107.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8,-13382.0,-1596.0,-10150.0,-114.0,334.0,-1.0,-13384.0,-1608.0,-10135.0,-136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,-13390.0,-1665.0,-10154.0,-32.0,423.0,-19.0,-13396.0,-1666.0,-10176.0,-76.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [357]:
bb = sample_dataframe.iloc[[0],:-1]
bb

,0,1,2,3,4,5,6,7,8,9,...,2270,2271,2272,2273,2274,2275,2276,2277,2278,2279
0,-13421.0,-1557.0,-10137.0,-44.0,362.0,-20.0,-13438.0,-1592.0,-10134.0,-37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [538]:
bb_c = pd.DataFrame(bb.values.reshape(380,-1),
                      index=bb.index.repeat(380))

In [539]:
bb_c

,0,1,2,3,4,5
0,-13421.0,-1557.0,-10137.0,-44.0,362.0,-20.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13429.0,-1614.0,-10105.0,-29.0,376.0,-38.0
0,-13429.0,-1614.0,-10105.0,-29.0,376.0,-38.0


In [564]:
bb_cc = bb_c.fillna(727199802211965)
bb_cc

,0,1,2,3,4,5
0,-1.342100e+04,-1.557000e+03,-1.013700e+04,-4.400000e+01,3.620000e+02,-2.000000e+01
0,-1.343800e+04,-1.592000e+03,-1.013400e+04,-3.700000e+01,3.630000e+02,-2.900000e+01
0,-1.343800e+04,-1.592000e+03,-1.013400e+04,-3.700000e+01,3.630000e+02,-2.900000e+01
0,-1.343800e+04,-1.592000e+03,-1.013400e+04,-3.700000e+01,3.630000e+02,-2.900000e+01
0,-1.343800e+04,-1.592000e+03,-1.013400e+04,-3.700000e+01,3.630000e+02,-2.900000e+01
0,-1.343800e+04,-1.592000e+03,-1.013400e+04,-3.700000e+01,3.630000e+02,-2.900000e+01
0,-1.343800e+04,-1.592000e+03,-1.013400e+04,-3.700000e+01,3.630000e+02,-2.900000e+01
0,-1.343800e+04,-1.592000e+03,-1.013400e+04,-3.700000e+01,3.630000e+02,-2.900000e+01
0,-1.342900e+04,-1.614000e+03,-1.010500e+04,-2.900000e+01,3.760000e+02,-3.800000e+01
0,-1.342900e+04,-1.614000e+03,-1.010500e+04,-2.900000e+01,3.760000e+02,-3.800000e+01


In [561]:
bb_ccc = bb_cc.replace(727199802211965, np.nan).dropna()

In [562]:
bb_ccc

,0,1,2,3,4,5
0,-13421.0,-1557.0,-10137.0,-44.0,362.0,-20.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13438.0,-1592.0,-10134.0,-37.0,363.0,-29.0
0,-13429.0,-1614.0,-10105.0,-29.0,376.0,-38.0
0,-13429.0,-1614.0,-10105.0,-29.0,376.0,-38.0


In [553]:
sad = sample_dataframe.copy()
sad

,0,1,2,3,4,5,6,7,8,9,...,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280
0,-13421.0,-1557.0,-10137.0,-44.0,362.0,-20.0,-13438.0,-1592.0,-10134.0,-37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,-13379.0,-1675.0,-10187.0,-25.0,379.0,-36.0,-13367.0,-1662.0,-10121.0,-38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,-13362.0,-1695.0,-10161.0,-55.0,426.0,-7.0,-13383.0,-1700.0,-10162.0,-49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,-13144.0,-1888.0,-10396.0,-75.0,424.0,2.0,-13136.0,-1877.0,-10397.0,-77.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,-13108.0,-1612.0,-10541.0,-130.0,325.0,1.0,-13108.0,-1612.0,-10541.0,-130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,-13148.0,-2318.0,-10280.0,-38.0,378.0,12.0,-13144.0,-2319.0,-10308.0,-57.0,...,-127.0,381.0,23.0,-13331.0,-2095.0,-10071.0,-127.0,381.0,23.0,1.0
6,-13366.0,-1200.0,-10235.0,15.0,428.0,-17.0,-13366.0,-1200.0,-10235.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
7,-13430.0,-1341.0,-10081.0,15.0,470.0,-28.0,-13404.0,-1354.0,-10107.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8,-13382.0,-1596.0,-10150.0,-114.0,334.0,-1.0,-13384.0,-1608.0,-10135.0,-136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,-13390.0,-1665.0,-10154.0,-32.0,423.0,-19.0,-13396.0,-1666.0,-10176.0,-76.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [563]:
sad = sad.fillna(727199802211965)
sad

,0,1,2,3,4,5,6,7,8,9,...,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280
0,-13421.0,-1557.0,-10137.0,-44.0,362.0,-20.0,-13438.0,-1592.0,-10134.0,-37.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
1,-13379.0,-1675.0,-10187.0,-25.0,379.0,-36.0,-13367.0,-1662.0,-10121.0,-38.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
2,-13362.0,-1695.0,-10161.0,-55.0,426.0,-7.0,-13383.0,-1700.0,-10162.0,-49.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
3,-13144.0,-1888.0,-10396.0,-75.0,424.0,2.0,-13136.0,-1877.0,-10397.0,-77.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
4,-13108.0,-1612.0,-10541.0,-130.0,325.0,1.0,-13108.0,-1612.0,-10541.0,-130.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
5,-13148.0,-2318.0,-10280.0,-38.0,378.0,12.0,-13144.0,-2319.0,-10308.0,-57.0,...,-1.270000e+02,3.810000e+02,2.300000e+01,-1.333100e+04,-2.095000e+03,-1.007100e+04,-1.270000e+02,3.810000e+02,2.300000e+01,1.0
6,-13366.0,-1200.0,-10235.0,15.0,428.0,-17.0,-13366.0,-1200.0,-10235.0,15.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
7,-13430.0,-1341.0,-10081.0,15.0,470.0,-28.0,-13404.0,-1354.0,-10107.0,4.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
8,-13382.0,-1596.0,-10150.0,-114.0,334.0,-1.0,-13384.0,-1608.0,-10135.0,-136.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
9,-13390.0,-1665.0,-10154.0,-32.0,423.0,-19.0,-13396.0,-1666.0,-10176.0,-76.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0


In [584]:
ddd = pd.DataFrame(sad.iloc[[0],:-1].values)
pd.DataFrame(ddd.values.reshape(380,-1),
                      index=ddd.index.repeat(380))

,0,1,2,3,4,5
0,-1.342100e+04,-1.557000e+03,-1.013700e+04,-4.400000e+01,3.620000e+02,-2.000000e+01
0,-1.343800e+04,-1.592000e+03,-1.013400e+04,-3.700000e+01,3.630000e+02,-2.900000e+01
0,-1.343800e+04,-1.592000e+03,-1.013400e+04,-3.700000e+01,3.630000e+02,-2.900000e+01
0,-1.343800e+04,-1.592000e+03,-1.013400e+04,-3.700000e+01,3.630000e+02,-2.900000e+01
0,-1.343800e+04,-1.592000e+03,-1.013400e+04,-3.700000e+01,3.630000e+02,-2.900000e+01
0,-1.343800e+04,-1.592000e+03,-1.013400e+04,-3.700000e+01,3.630000e+02,-2.900000e+01
0,-1.343800e+04,-1.592000e+03,-1.013400e+04,-3.700000e+01,3.630000e+02,-2.900000e+01
0,-1.343800e+04,-1.592000e+03,-1.013400e+04,-3.700000e+01,3.630000e+02,-2.900000e+01
0,-1.342900e+04,-1.614000e+03,-1.010500e+04,-2.900000e+01,3.760000e+02,-3.800000e+01
0,-1.342900e+04,-1.614000e+03,-1.010500e+04,-2.900000e+01,3.760000e+02,-3.800000e+01


In [608]:
def get_dataframe_column_for_comparison(df):
    df = pd.DataFrame(df).T
    dfc = pd.DataFrame(df.values.reshape(380,-1),
                      index=df.index.repeat(380))
    dfc = dfc.replace(727199802211965, np.nan).dropna()
    return dfc.values

In [609]:
def run_dtw(df1, df2):
    dist, cost, acc, path = dtw(get_dataframe_column_for_comparison(df1), get_dataframe_column_for_comparison(df2), dist=lambda x, y: norm(x - y, ord=1))
    return dist

In [612]:
X = sad.iloc[:, :-1].values#sample_dataframe.iloc[:, :-1].values
y = sad.iloc[:, -1].values#sample_dataframe.iloc[:, -1].values

In [613]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42, stratify=y)

In [614]:
knn = KNeighborsClassifier(n_neighbors=1, metric=run_dtw)

In [615]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30,
           metric=<function run_dtw at 0x11be00b70>, metric_params=None,
           n_jobs=None, n_neighbors=1, p=2, weights='uniform')

In [ ]:
train_accuracy[i] = knn.score(X_train, y_train)

In [ ]:
#Compute accuracy on the test set
test_accuracy[i] = knn.score(X_test, y_test) 

In [610]:
scores = cross_val_score(knn, X, y)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


KeyboardInterrupt: 